<a href="https://colab.research.google.com/github/NursimaErkoc/CNN_ile_cicek_siniflandirma/blob/main/CNN_cicek_t%C3%BCr%C3%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
flowers dataset :
rgb: 224x224
CNN ile siniflandirma modeli oluşturma ve problemi cözme
"""

#import libraries
!pip install tensorflow matplotlib tensorflow_datasets

from tensorflow_datasets import load # veri seti yükleme
from tensorflow.data import AUTOTUNE # veri seti optimizasyonu
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import(
    Conv2D, # 2D convolutional layer
    MaxPooling2D, # max pooling layer
    Flatten, # çok boyutlu veriyi tek boyutlu hale getirme düzleştirme
    Dense, # fully connected layer tam bağlantılı katman
    Dropout  #rastgele noronları kapatma ve overfittingi engelleme
    )
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    EarlyStopping, #erken durdurma
    ReduceLROnPlateau, # ogrenme oranını azaltma
    ModelCheckpoint # model kaydetme
)

import tensorflow as tf
import matplotlib.pyplot as plt




In [ ]:
(ds_train, ds_val), ds_info=load(
    "tf_flowers" ,# veri seti ismi
    split = ["train[:80%]", # veri setinin %80i eğitim için
             "train[80%:]"], # veri setinin kalan % 20 si test için (80yüzdesinden sonraki kısım )
    as_supervised=True, # veri setinin gorsel etiket cıftının olması
    with_info=True # veri seti hakkında bilgi alma
)
print (ds_info.features) # veri seti hakkımda bilgi yazdirma
print ("Number of classes", ds_info.features["label"].num_classes)


In [ ]:
#ornek veri görselleştirme
# eğitim setinde rastgele 3 resim ve etiket alma
fig = plt.figure(figsize=(10,5))
for i, (image,label ) in enumerate(ds_train.take(3)):
  ax = fig.add_subplot(1,3,i+1)
  ax.imshow(image.numpy().astype("uint8")) #resim görselleştirme
  ax.set_title(f"Etiket : {label.numpy()}") # etiketi başlık olarak yzadırma
  ax.axis("off") # eksenleri kapatma

plt.tight_layout()# orantılı görselleştirme
plt.show()# grafiği gösterme



In [ ]:

IMG_SIZE = (180, 180)

# data augmentation + preprocessing
def preprocess_train(image, label):
    """
    resize
    random flip
    brightness
    contrast
    crop
    normalize
    """
    image = tf.image.resize(image, IMG_SIZE) # boyutlandırma
    image = tf.image.random_flip_left_right(image) # yatay olarak cevirme
    image = tf.image.random_brightness(image, max_delta=0.1) # rastgele parlaklık
    image = tf.image.random_contrast(image, lower=0.9, upper=1.2) # rastgele kontrast
    image = tf.image.random_crop(image, size=(160, 160, 3)) # rastgele kırpma
    image = tf.image.resize(image, IMG_SIZE) #tekrar boyutlandırma

    image = tf.cast(image, tf.float32) / 255.0 # normalize etme
    return image, label


def preprocess_val(image, label):
    """
    resize + normalize
    (validation/test setine augmentation uygulanmaz)
    """
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32)/255.0
    return image, label


# train dataset
ds_train = (
    ds_train
    .map(preprocess_train, num_parallel_calls=AUTOTUNE) # ön işleme ve agumentasyon
    .shuffle(1000) # kariştirma
    .batch(32)# batch boyutu
    .prefetch(AUTOTUNE) # veri setini önceden hazırlamak
)

# validation dataset
ds_val = (
    ds_val
    .map(preprocess_val, num_parallel_calls=AUTOTUNE) # ön işleme
    .batch(32)
    .prefetch(AUTOTUNE)
)


In [ ]:
# CNN modelini oluşturma
model= Sequential ([

    # Feature Extraction Layers
    Conv2D(32,(3,3), activation= "relu" , input_shape= (*IMG_SIZE,3)), #32 filtre, 3x3 karnel,relu aktivasyon 3 kanalRGB
    MaxPooling2D((2,2)), # 2x2 max pooling

    Conv2D(64,(3,3), activation= "relu"), # 64 filtre 3x3 karnel relu aktivasyon
    MaxPooling2D((2,2)) ,

    Conv2D(128,(3,3), activation= "relu"),
    MaxPooling2D((2,2)),

    # Classification Layers
    Flatten(), # çok boyutlu veriyi tek boyutlu hale getirme
    Dense(128, activation="relu"),
    Dropout(0.5), # overfittingi enngellemek için
    Dense(ds_info.features["label"].num_classes, activation = "softmax" )  # çıkıs katmanı softmax aktivasyonu
])

    # 3x3 kernel kaydırma penceresi küçük olması komşu pikseller arasındaki ince bağlantıyı
    # yeterli ve hasaplama açısından verimli ardışık 3x3 ler tek bir büyüğe 5x5 denk gelebilecek daha geniş bir perspektif sunar
    # art arda 3x3 kullanmak hem paremetre sayısını hemde hesaplama yükünü azaltıyor detaylara erişmek isityorsak küçük kullanacağız

    # Ağ derinleştikçe tespit edilebilecek özellikler karmaşıklaşıyor
    # hiyerarşik öğrenme söz konusu ilk başta kenarları ayırtedebilirken derinlesştikçe
    # yaprak kıvrımını tespit edebiliyor bu nedenle filtre sayısı arttırlıyor

    # Maxpoolingte 2x2 geniş alana bakarken hesap yükünü azaltmak ve fazladan ayrıntılardan kurtulmak için yapılıyor
    # uzaysal boyutları küçültüp katmanların derinlik boyutunu arttırıyoruz 32 64 128
    # modelin karmaşıklığını arttırıken maxpooling ile karmaşık azaltmak çelişiyor gibi görünsede
    # paremetre sayısını makul tutarken temsil gücü korunur
    # Maxpooling kullanılmazsa model çok yavaş çalışır
    # Hesap kapasite ve ifade gücü dengesi kurulmaya çalışılıyor

    # 32,64,128 sırasıyla giderken ilk katman çizgileri öğenirken son katman ince ayrıntıları öğrenmeye çalışıyor






In [ ]:
# callbacks
callbacks= [
    # eger val loss 3 epoch boyunca iyileşmezse eğitimi durdur ve en iyi ağırlıkları yükle
    EarlyStopping(monitor= "val_loss", patience=3 , restore_best_weights = True ) ,

    # val loss 2 epoch boyuncaiyileşmezse learning rate 0.2 carpani ile azaltılıyor lr azaltma daha küçük adımlarla daha çok öğrenme gerçekleşir
    ReduceLROnPlateau(monitor="vall_loss", factor =0.2 , patience = 2, verbose =1 , min_lr= 1e-9), # öğrenme oranını azalatma

    #her epoch sonunda eğer model daha iyi ise kaydolur
    ModelCheckpoint("best_model.h5", save_best_only= True ) # model kaydetme en iyi modeli kaydet
]


In [ ]:
#derleme
model.compile(
    optimizer= Adam(learning_rate=0.001) ,
    loss= "sparse_categorical_crossentropy", # kayıp fonksiyonu , etiketler tamsayi olduugu icin sparse kullan
    metrics = ["accuracy"]
)
print (model.summary()) # model özeti



In [ ]:
#traning
history= model.fit(
    ds_train, # eğitim  veri seti
    validation_data= ds_val,
    epochs=10,
    callbacks= callbacks,
    verbose=1 # eğitim ilerlemesini göster
)


In [ ]:
# model evaluation
plt.figure(figsize=(12,5))

# doğruluk grafiği
plt.subplot(1,2,1)
plt.plot(history.history["accuracy"], label="Egitim Dogrulugu")
plt.plot(history.history["val_accuracy"], label= "Validasyon Dogrulugu")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title ("Model Accuracy")
plt.legend()

#loss plot
plt.subplot(1,2,2)
plt.plot(history.history["loss"], label= "Egitim Kaybi")
plt.plot(history.history["val_loss"], label="Validasyon Kaybi")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Model Loss")
plt.legend()
plt.tight_layout()
plt.show()

